In [1]:
import json
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from datetime import datetime

with open('subgraph_Thomas Carson_3_degrees.json', 'r') as file:
    data = json.load(file)

nodes = data['nodes']
links = data['links']

links_df = pd.DataFrame(links)
links_df['start_date'] = pd.to_datetime(links_df['start_date'])
links_df['end_date'] = pd.to_datetime(links_df['end_date'], errors='coerce')


links_df['end_date'] = links_df['end_date'].fillna(pd.Timestamp('2036-12-31'))



def simple_pagerank(G, alpha=0.85, max_iter=100, tol=1.0e-6):
    nodes = list(G.nodes())
    N = len(nodes)
    if N == 0:
        return {}

    pagerank = {node: 1.0 / N for node in nodes}
    
    for _ in range(max_iter):
        prev_pagerank = pagerank.copy()
        for node in nodes:
            rank_sum = 0
            for pred in G.predecessors(node):
                rank_sum += prev_pagerank[pred] / len(list(G.successors(pred)))
            pagerank[node] = (1 - alpha) / N + alpha * rank_sum
        
        err = sum(abs(pagerank[node] - prev_pagerank[node]) for node in nodes)
        if err < tol:
            break

    return pagerank

time_range = pd.date_range(start='2032-01-01', end='2036-12-31', freq='M')

pagerank_scores = {node['id']: [] for node in nodes}



for current_time in time_range:
    G = nx.DiGraph()
    G.add_nodes_from([node['id'] for node in nodes])
    
    active_links = links_df[(links_df['start_date'] <= current_time) & (links_df['end_date'] >= current_time)]
    for _, link in active_links.iterrows():
        G.add_edge(link['source'], link['target'])
    
    pagerank = nx.pagerank(G)
    
    for node_id in pagerank_scores:
        pagerank_scores[node_id].append(pagerank.get(node_id, 0))

pagerank_df = pd.DataFrame(pagerank_scores, index=time_range)



/var/folders/5x/xz2xn6k92fzfkfdcl0qfcnsw0000gn/T/ipykernel_2508/1572751527.py:44: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  time_range = pd.date_range(start='2032-01-01', end='2036-12-31', freq='M')


In [2]:
import ipywidgets as widgets
from IPython.display import display


def update_plot(selected_nodes):
    fig, ax = plt.subplots(figsize=(14, 8))
    for node_id in selected_nodes:
        ax.plot(pagerank_df.index, pagerank_df[node_id], label=node_id)
    ax.set_title(f'PageRank Scores Over Time for Selected Nodes')
    ax.set_xlabel('Time')
    ax.set_ylabel('PageRank Score')
    ax.legend(title='Nodes')
    #plt.axvline(pd.to_datetime('2035-09-02'), color = 'red', linestyle = '--', label = 'transfer day')
    plt.tight_layout()
    plt.show()


node_multiselect = widgets.SelectMultiple(
    options=pagerank_df.columns,
    description='Nodes:',
    disabled=False,
)


widgets.interact(update_plot, selected_nodes=node_multiselect)

interactive(children=(SelectMultiple(description='Nodes:', options=('Fox, Chapman and Stone', 'Graham-Andrews'…

<function __main__.update_plot(selected_nodes)>